# Detección de fallos por imagen

## Imports

In [ ]:
# ============================================================
# 0. Imports básicos
# ============================================================
import os
import shutil
import json
import pickle
from google.colab import drive
from tqdm import tqdm
from ultralytics import YOLO


## 1 Limpiar y montar Google Drive

In [ ]:
# ============================================================
# 1. Limpiar y montar Google Drive
# ============================================================

print("--- 1. Limpiando el Punto de Montaje ---")

# Limpiar posible montaje previo
if os.path.exists('/content/drive'):
    try:
        shutil.rmtree('/content/drive')
        print("✅ Carpeta /content/drive eliminada con éxito.")
    except Exception as e:
        print(f"❌ Error al limpiar la carpeta /content/drive: {e}")
        pass

# Recrear carpeta vacía
os.makedirs('/content/drive', exist_ok=True)
print("✅ Carpeta /content/drive recreada y vacía.")

# Montar Google Drive
print("\n--- 2. Montando Google Drive ---")
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive montado exitosamente.")
except Exception as e:
    print(f"❌ ERROR CRÍTICO DE MONTAJE: {e}")


## 2 Rutas


In [ ]:
# ============================================================
# 2. Instalación de dependencias y definición de rutas
# ============================================================

# Instalar Ultralytics YOLO
!pip install -q ultralytics

# Ruta principal del proyecto en Drive
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/train-wheel-defect-detection'
DATA_PATH = os.path.join(DRIVE_PROJECT_PATH, 'data/img/')

TRAIN_DIR = os.path.join(DATA_PATH, 'train')
VALID_DIR = os.path.join(DATA_PATH, 'valid')

print(f"Ruta del proyecto configurada como: {DRIVE_PROJECT_PATH}")

print("--- Diagnóstico de Archivos en Drive ---")
print(f"Buscando en: {TRAIN_DIR}")
!ls -R {TRAIN_DIR}

print("-" * 40)
print(f"Buscando en: {VALID_DIR}")
!ls -R {VALID_DIR}


## 3 Conversion de COCO a YOLO

In [ ]:
# ============================================================
# 3. Conversión COCO → YOLO (generar labels)
# ============================================================

# Mapeo COCO → YOLO
ID_MAP = {1: 0, 2: 1, 3: 2, 4: 3}
JSON_NAMES = ['_annotations.coco.json', 'annotations.coco.json']  # posibles nombres

def convert_coco_to_yolo_safe(split_name, data_path, id_map):
    """Convierte anotaciones COCO a formato YOLO, buscando varios nombres de JSON."""
    print(f"\n--- Procesando datos de {split_name} ---")
    data = None

    # 1. Intentar cargar el JSON con los nombres posibles
    for name in JSON_NAMES:
        path_to_try = os.path.join(data_path, split_name, name)
        try:
            with open(path_to_try, 'r') as f:
                data = json.load(f)
                print(f"✅ JSON de {split_name} cargado correctamente usando: {name}")
                break
        except FileNotFoundError:
            continue

    if data is None:
        print(f"❌ ERROR CRÍTICO: No se pudo encontrar el archivo JSON en {split_name}.")
        return

    # 2. Crear carpeta labels (misma altura que images)
    img_id_map = {
        img['id']: {
            'file_name': img['file_name'],
            'width': img['width'],
            'height': img['height']
        }
        for img in data['images']
    }

    labels_dir = os.path.join(data_path, split_name, 'labels')
    os.makedirs(labels_dir, exist_ok=True)

    annotations_by_file = {}

    # 3. Procesar anotaciones COCO → YOLO
    for ann in tqdm(data['annotations'], desc=f"Generando anotaciones para {split_name}"):
        if ann['category_id'] not in id_map:
            continue

        img_info = img_id_map[ann['image_id']]
        file_name = img_info['file_name']
        yolo_class_id = id_map[ann['category_id']]

        x_min, y_min, w_abs, h_abs = ann['bbox']
        W, H = img_info['width'], img_info['height']

        # Conversión COCO [x_min, y_min, w, h] → YOLO [cx, cy, w, h] normalizado [web:2][web:5][web:8]
        x_center = (x_min + w_abs / 2) / W
        y_center = (y_min + h_abs / 2) / H
        w_norm = w_abs / W
        h_norm = h_abs / H

        yolo_line = f"{yolo_class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"

        if file_name not in annotations_by_file:
            annotations_by_file[file_name] = []
        annotations_by_file[file_name].append(yolo_line)

    # 4. Escribir archivos .txt
    count_written = 0
    for file_name, lines in annotations_by_file.items():
        label_filename = file_name.rsplit('.', 1)[0] + '.txt'
        output_path = os.path.join(labels_dir, label_filename)
        with open(output_path, 'w') as f:
            f.write('\n'.join(lines))
        count_written += 1

    print(f"✅ Etiquetas YOLO creadas exitosamente para {split_name}: {count_written} archivos")


# Ejecutar conversión para train y valid
convert_coco_to_yolo_safe('train', DATA_PATH, ID_MAP)
convert_coco_to_yolo_safe('valid', DATA_PATH, ID_MAP)

print("\n--- ¡Terminado! Archivos .txt generados en las carpetas 'labels'. ---")


## 4 Creacion del .yaml

In [ ]:
# ============================================================
# 4. Generar archivo YAML para YOLOv8
# ============================================================

YAML_FILENAME = 'rueda_tren_data.yaml'
CLASS_NAMES = ['Cracks-Scratches', 'Discoloration', 'Shelling', 'Wheel']

# YOLOv8 espera:
# path: carpeta raíz del dataset
# train: subcarpeta train
# val: subcarpeta valid [web:4][web:7][web:10]
yaml_content = f"""
# Archivo de configuración para YOLOv8
path: {DRIVE_PROJECT_PATH}/data/img/
train: train
val: valid

nc: {len(CLASS_NAMES)}
names: {CLASS_NAMES}
"""

local_yaml_path = f'/content/{YAML_FILENAME}'
drive_yaml_path = os.path.join(DRIVE_PROJECT_PATH, YAML_FILENAME)
project_save_dir = os.path.join(DRIVE_PROJECT_PATH, 'runs/detect')

try:
    # Guardar YAML local y copiar a Drive
    with open(local_yaml_path, 'w') as f:
        f.write(yaml_content)

    !mkdir -p {DRIVE_PROJECT_PATH}
    !cp {local_yaml_path} {drive_yaml_path}

    print(f"✅ YAML de configuración creado en Drive: {drive_yaml_path}")
except Exception as e:
    print(f"❌ ERROR al generar el YAML: {e}")


## 5 Entrenamiento del modelo preentrenado 

In [ ]:
# ============================================================
# 5. Entrenamiento del modelo YOLOv8
# ============================================================

print("\n--- Iniciando Entrenamiento YOLOv8 ---")
print(f"Verificación: Buscando imágenes en {DRIVE_PROJECT_PATH}/data/img/train/images")
print("-" * 40)

# Cargar modelo base YOLOv8
model = YOLO('yolov8n.pt')

# Entrenar modelo usando el YAML generado [web:7][web:10]
results = model.train(
    data=drive_yaml_path,
    epochs=50,
    imgsz=640,
    batch=-1,
    name='yolov8_defect_detection_v1',
    project=project_save_dir
)

print("\n--- Proceso de entrenamiento iniciado/executado exitosamente ---")


## 6 Pickle

In [ ]:
# ============================================================
# 6. Exportar modelo entrenado a Pickle (.pkl)
# ============================================================

DRIVE_PROJECT_PATH = '/content/drive/MyDrive/train-wheel-defect-detection'
MODEL_PATH = os.path.join(
    DRIVE_PROJECT_PATH,
    'runs/detect/yolov8_defect_detection_v1/weights/best.pt'
)
PKL_OUTPUT_PATH = os.path.join(DRIVE_PROJECT_PATH, 'yolov8_detector_export.pkl')

# Cargar modelo entrenado
try:
    model_to_save = YOLO(MODEL_PATH)
except FileNotFoundError:
    print("❌ ERROR: El archivo 'best.pt' no se encontró. No se puede serializar.")
else:
    print(f"\n--- Exportando a .pkl (Pickle) ---")
    try:
        with open(PKL_OUTPUT_PATH, 'wb') as f:
            pickle.dump(model_to_save, f)
        print(f"✅ Modelo guardado como Pickle en: {PKL_OUTPUT_PATH}")
    except Exception as e:
        print(f"❌ ERROR al guardar como Pickle: {e}")


## 7 ONNX

In [ ]:
# ============================================================
# 7. Exportar modelo entrenado a ONNX
# ============================================================

DRIVE_PROJECT_PATH = '/content/drive/MyDrive/train-wheel-defect-detection'
MODEL_PATH = os.path.join(
    DRIVE_PROJECT_PATH,
    'runs/detect/yolov8_defect_detection_v1/weights/best.pt'
)
ONNX_OUTPUT_PATH = os.path.join(DRIVE_PROJECT_PATH, 'yolov8_detector.onnx')

# Cargar modelo entrenado
try:
    model = YOLO(MODEL_PATH)
    print(f"✅ Modelo cargado desde: {MODEL_PATH}")
except FileNotFoundError:
    print("❌ ERROR: El archivo 'best.pt' no se encontró. Asegúrese de que el entrenamiento ha finalizado.")
else:
    print("\n--- Exportando a ONNX (Recomendado para despliegue) ---")
    try:
        # Ultralytics permite exportar directamente a ONNX [web:3][web:6][web:9]
        path = model.export(format='onnx')
        print(f"✅ Modelo exportado exitosamente a ONNX en: {path}")
    except Exception as e:
        print(f"❌ ERROR al exportar a ONNX: {e}")
